In [2]:
from dotenv import load_dotenv
import pandas as pd
from datetime import date, datetime
from src.model.repositorios.xls.movimentacao_estoque import MovimentacaoDeEstoqueXls

In [3]:
load_dotenv()

True

In [4]:
mov_estoque = MovimentacaoDeEstoqueXls()

In [5]:
dt_inicial = datetime(2024, 1, 1)
dt_final = datetime(2024, 8, 30)

mov = mov_estoque.buscar_movimentacao(dt_inicial=dt_inicial, dt_final=dt_final)

In [16]:
df = pd.read_csv('data.csv')

df['descricao'] = df['descricao'].str.capitalize()
df['marca'] = df['marca'].str.capitalize()
df['data_movimento'] = pd.to_datetime(df['data_movimento'], format='%d/%m/%Y %H:%M:%S')
df['valor_total'] = df['qtd_negociada'] * (df['vlr_unitario'] - df['vlr_desconto'])

df

,nro_documento,tipo_movimento,cod_parceiro,data_movimento,cod_produto,descricao,marca,qtd_negociada,vlr_unitario,vlr_desconto,valor_total
0,1563158,P,242,2024-09-02 08:30:18,45251,Impermeabilizante tecplus top 18kg,Quartzolit,30.0,51.65,0.00,1549.50
1,1563158,P,242,2024-09-02 08:30:18,45249,Impermeabilizante tecplus 18l,Quartzolit,10.0,79.18,0.00,791.80
2,1563158,P,242,2024-09-02 08:30:18,45608,Pintura asfaltica 18l b.solvente,Quartzolit,4.0,289.34,0.00,1157.36
3,1563158,P,242,2024-09-02 08:30:18,45252,Impermeabilizante tecplus top 4kg,Quartzolit,5.0,21.28,0.00,106.40
4,1563158,P,242,2024-09-02 08:30:18,45241,Aditivo plastificante pt 18l,Quartzolit,6.0,137.12,0.00,822.72
...,...,...,...,...,...,...,...,...,...,...,...
95424,1572837,D,8338,2024-09-16 08:02:00,10485,Soquete adap porc. e-40 p/ e-27 4a biv,Decorlux,50.0,3.67,0.37,165.00
95425,1567415,D,5206,2024-09-06 13:38:01,44069,Lampada led alta pot. 30w bivolt 6500k,Avant,3.0,9.25,0.37,26.64
95426,1565579,D,590,2024-09-04 14:06:32,28883,"Boia automatica 16a cbs-1,2m(30)",Soprano,1.0,27.88,1.12,26.76
95427,1564057,D,11695,2024-09-03 08:11:47,45612,Escada 7 degraus domestica alum.,Alumasa,1.0,230.91,9.24,221.67


In [6]:
columns_names: set = {
        'nro_documento',
        'tipo_movimento',
        'cod_parceiro',
        'data_movimento',
        'cod_produto',
        'descricao',
        'marca',
        'qtd_negociada',
        'vlr_unitario',
        'vlr_desconto'
    }

In [ ]:
{item: item in df.columns for item in columns_names}

In [ ]:
df[(df['tipo_movimento'] == 'V') & (df['data_movimento'].between(datetime(2024,9,14), datetime(2024,9,16), inclusive='neither'))].sort_values('data_movimento', ascending=False)

In [7]:
#df_marca = df[df['tipo_movimento'] == 'V'].groupby(['marca'])[['qtd_negociada', 'vlr_unitario']].sum().reset_index()

df_marca = df[(df['tipo_movimento'] == 'V') & (df['data_movimento'].between(datetime(2024,9,15), datetime(2024,9,17)))]\
    .groupby(['marca'])\
    .agg({'nro_documento': 'nunique', 'qtd_negociada': 'sum', 'vlr_unitario': 'sum'})\
    .sort_values('vlr_unitario', ascending=False)\
    .reset_index()

df_marca['vlr_unitario'] = df_marca['vlr_unitario'].apply(lambda x: round(x, 2))
df_marca

,marca,nro_documento,qtd_negociada,vlr_unitario
0,Cobrecom,3,6.0,2735.00
1,Boreal,2,3.0,1640.33
2,Ciser,6,309.0,769.19
3,Otto,1,2.0,559.51
4,Plastimax,2,7.0,497.76
...,...,...,...,...
87,Trio eletr,1,3.0,5.14
88,Firlon,1,5.0,4.69
89,Incob,1,10.0,3.07
90,Rip-rebouç,2,175.0,2.12


### Produtos

In [26]:
df_produtos = df[(df['tipo_movimento'] == 'V') & (df['data_movimento'].between(datetime(2024,9,1), datetime(2024,9,15)))]\
    .groupby(['cod_produto', 'descricao', 'marca'])\
    .agg({'nro_documento': 'nunique', 'qtd_negociada': 'sum', 'valor_total': 'sum'})\
    .sort_values('valor_total', ascending=False)\
    .reset_index()

df_produtos['perc_vendas'] = df_produtos[['valor_total']] / df_produtos[['valor_total']].sum()

total = round(df_produtos[['valor_total']].sum(), 2)
df_produtos['valor_total_acu'] = round(df_produtos['valor_total'].cumsum(), 2)

df_produtos['perc_vendas_acu'] = df_produtos[['perc_vendas']].cumsum()

df_produtos['curva_abc'] = ''

df_produtos

C:\Users\alisson\AppData\Local\Temp\ipykernel_1152\2490104885.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_produtos[df_produtos['perc_vendas_acu'] <= 0.20]['curva_abc'] = 'A'


,cod_produto,descricao,marca,nro_documento,qtd_negociada,valor_total,perc_vendas,valor_total_acu,perc_vendas_acu,curva_abc
0,22551,Caixa d agua plast. azul 1.000l,Fortlev,117,620.0,206139.35000,0.033226,206139.35,0.033226,
1,28660,Tubo esg 100mm pvc. 6m (5),Krona,70,1434.0,83088.12405,0.013392,289227.47,0.046619,
2,29548,Arame ovalado 16x14 nelore 1000m (1),Morlan,20,151.0,71847.74086,0.011581,361075.21,0.058199,
3,28648,Tubo sold 25mm pvc. 6m (8),Krona,91,4366.0,63765.89780,0.010278,424841.11,0.068477,
4,22549,Caixa d agua plast. azul 500l.,Fortlev,63,283.0,60949.51000,0.009824,485790.62,0.078301,
...,...,...,...,...,...,...,...,...,...,...
6344,40053,Arruela lisa 7/16 polido (2kg) +-198 un,Sm,1,50.0,0.00000,0.000000,6204127.01,1.000000,
6345,41815,Espatula de aco 10cm cabo plast,Atlas,1,17.0,0.00000,0.000000,6204127.01,1.000000,
6346,40018,Plafon emb. quad. 30x30 led 24w br,Avant,1,1.0,0.00000,0.000000,6204127.01,1.000000,
6347,17973,Desemp. plast 18x30cm estriada (12).,Compel,1,56.0,0.00000,0.000000,6204127.01,1.000000,
